In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import os
import xlwings as xw
from tqdm.notebook import tqdm

In [2]:
def availablef():
    df = pd.read_excel("alloc_source_file.xlsx",sheet_name='available')
    return df
def nd():
    dfn = pd.read_excel("alloc_source_file.xlsx",sheet_name='needed')
    return dfn    

In [3]:
available = availablef()
needed=nd()
dfn=nd()

### Checking the Needed Tab for Correct Format

In [4]:
# Check that the expected columns are present
expected_cols=['craft','quantity','price','craftprice']
if not all(col in dfn.columns for col in expected_cols):
    raise ValueError("Columns not found in DataFrame: {}".format(expected_cols))
    
# checking if 'quantity' and 'price' columns are numeric
for col in ['quantity','price']:
    if dfn[col].dtype != 'float64' and dfn[col].dtype !='int64':
        raise TypeError("Column '{}' is not numeric ".format(col))
        
# Check that 'craftprice' is a conatenation of 'craft' and 'price'
for index,row in dfn.iloc[2:].iterrows():
    if str(row['craft'])+str(row['price'])!= str(row['craftprice']):
        raise ValueError("Row {} does not match concatenation of 'craft' and 'price'".format(index))
        
# Confirm that the changes have been moade
print("Needed column is correct")

Needed column is correct


### Check if the craftprice is present commonly between needed and available

In [5]:
is_present= needed['craftprice'].isin(available['craft']+available['price'].astype(str))
if is_present.any():
    print("At least one value in needed['craftprice'] is also present in available['craftprice'],can be allocated")
else:
    print("No values in needed['craftprice'] are present in available['craftprice'],hence cannot be allocated")

At least one value in needed['craftprice'] is also present in available['craftprice'],can be allocated


In [19]:
def allocation_engine(pcs,brand,Style_Repeats_Allowed):
    global availablef
    global original_needed
    global newfile
    available=availablef()
    needed = nd()
    
    available=available[available['quantity']>0]

    available=available[available['Brand Type']==Brand]


    originally_available = available.copy()
    oa_agg=pd.DataFrame(originally_available.groupby(['craft','price'])['quantity'].sum()).reset_index()
    available=available.sample(frac=1).reset_index(drop=True)
    available1=available.groupby('Style Code').head(Style_Repeats_Allowed)
    df_both = pd.concat([available1,available],ignore_index=True) 
    df_dif=df_both.drop_duplicates(keep=False)
    
    

    available=available1.copy()
    available['craftprice']=available['craft']+available['price'].astype(str)
    availcheck=available.groupby(['craftprice'])['quantity'].sum()
    availcheckd=pd.DataFrame(availcheck)
    availcheckdr=availcheckd.reset_index()





    original_needed=needed.copy()



    needed['quantity']=needed['quantity']*pcs 
    needed['quantity'] = needed['quantity'].astype (int)


    needed.sort_values(by='quantity',ascending=False)
    Original_quantity=needed.copy()
    



    allocated=available.copy()
    allocated['quantity']=0


    sufcheck=allocated.groupby(['craftprice'])['quantity'].sum()
    sufcheckd=pd.DataFrame(sufcheck)
    sufcheckdr=sufcheckd.reset_index()



    needed=needed.merge(sufcheckdr,on=['craftprice'])
    needed=needed.merge(availcheckdr,on=['craftprice'])

    t=0
    l=0


    
    for n in needed.index:
        s=0
        while needed.loc[n,'quantity_x']>0:
            for a in available.index:
                s=s+1
                t=t+1



                if needed.loc[n,'craftprice']==available.loc[a,'craftprice']:


                    if needed.loc[n,'quantity_x']!=0:


                        if needed.loc[n,'quantity_y']<=needed.loc[n,'quantity']:


                            if available.loc[a,'quantity']!=0:


                                if allocated.loc[a,'quantity']==0:


                                    allocated.loc[a,'quantity']=1+allocated.loc[a,'quantity']
                                    needed.loc[n,'quantity_x']=needed.loc[n,'quantity_x']-1
                                    needed.loc[n,'quantity_y']=needed.loc[n,'quantity_y']+1
                                    available.loc[a,'quantity']=available.loc[a,'quantity']-1
                                    l=l+1



            if needed.loc[n,'quantity_x']==0 or needed.loc[n,'quantity_y']==needed.loc[n,'quantity'] or s>=len(available.index):
                break
    print('======================')        
    available=pd.concat([df_dif,available],ignore_index=True)


    timestr=time.strftime("%d%m%Y-%H%M%S")










    target_file = f"allocated_{p}_{Brand}_{timestr}.xlsx"







    allocated.to_excel(target_file,index=False)

    print('Average MRP:',allocated['MRP'].mean())
    if not allocated.empty and allocated['Brand Type'].eq(allocated['Brand Type'].iloc[0]).all():
        print("Brand Name for all this allocation is ",allocated['Brand Type'][0])
    print('Available Qty in ',Brand,' Before Allocation:',originally_available.groupby(['craft','price'])['quantity'].sum().sum())
    print('Available Qty in ',Brand,' After Allocation: ',available.groupby(['craft','price'])['quantity'].sum().sum())
    print('Total Qty Required:',pcs)
    print('Total Qty Alloted:',allocated.groupby(['craft','price'])['quantity'].sum().sum())
    print('Max Qty Allocated per Style Code:',allocated.groupby('Style Code')['quantity'].sum().max())
    print('Max Qty Allocated:',allocated['quantity'].max())
    print('Min Qty Allocated:',allocated['quantity'].min())
    print('Min Qty available per Article in ',Brand,'Post Allocation: ',available['quantity'].min())
    print('\n Percentage Breakup...................')
    left = pd.DataFrame(available.groupby(['craft','price'])['quantity'].sum()).reset_index()
    Alloc=pd.DataFrame(allocated.groupby(['craft','price'])['quantity'].sum()).reset_index()
    Alloc['Alloc%']=Alloc['quantity']*100/Alloc.groupby(['craft','price'])['quantity'].sum().sum()
    AllocMerge= pd.merge(Alloc,Original_quantity,how='inner',on=['craft','price'])
    AllocMerge.rename(columns={'quantity_x':'alloc_qty','quantity_y':'req_qty'},inplace=True)
    AllocMerge2=pd.merge(AllocMerge,oa_agg,how='inner',on=['craft','price'])
    AllocMerge2.rename(columns={'quantity':'orig_qty'},inplace=True)
    AllocMerge3=pd.merge(AllocMerge2,left,how='inner',on=['craft','price'])
    AllocMerge3.rename(columns={'quantity':'left'},inplace=True)
    AllocMerge3['Deficit']=AllocMerge3['req_qty']-AllocMerge3['alloc_qty']
    print(AllocMerge3[['craft','price','orig_qty','req_qty','alloc_qty','Deficit','left','Alloc%']].sort_values(by=['craft','Alloc%'],ascending=False))

    available=available.reset_index(drop=True)
    newfile="available_"+p+Brand+timestr+".xlsx"


    with pd.ExcelWriter(newfile) as writer:
        available.to_excel(writer, sheet_name='available', index=False)
        original_needed.to_excel(writer, sheet_name='needed', index=False)
    print("\n=====Post Allocation Craft Break-up======")
    print(available.groupby('craft')['quantity'].sum()*100/available.groupby('craft')['quantity'].sum().sum())
    print("\n=====Originally needed==================")
    print(original_needed[['craft','quantity','price']])
    print("==========================================")
    print("available_"+p+Brand+timestr+".xlsx")


In [22]:
pcs= 300
Brand = "Brand_C"
p = 'Cuddalore' # Name of store
Style_Repeats_Allowed=1
allocation_engine(pcs,Brand,Style_Repeats_Allowed)

Average MRP: 15158.245937424206
Brand Name for all this allocation is  Brand_C
Available Qty in  Brand_C  Before Allocation: 30305
Available Qty in  Brand_C  After Allocation:  30223
Total Qty Required: 300
Total Qty Alloted: 82
Max Qty Allocated per Style Code: 1
Max Qty Allocated: 1
Min Qty Allocated: 0
Min Qty available per Article in  Brand_C Post Allocation:  0

 Percentage Breakup...................
      craft  price  orig_qty  req_qty  alloc_qty  Deficit  left     Alloc%
4  Craft_99      2       124       60         15       45   109  18.292683
3  Craft_57      3       139       15         15        0   124  18.292683
2  Craft_49      2       188      120         24       96   164  29.268293
1   Craft_3      2       112       15         15        0    97  18.292683
0   Craft_1      2        93       90         13       77    80  15.853659

=====Post Allocation Craft Break-up======
craft
Craft_1      0.658439
Craft_10     0.549251
Craft_100    0.903286
Craft_101    0.764319
Craf